# AI Agent with Tools and Memory

AI agents are an integral part of our lives ranging from applications like Customer service, automation of business tasks, stock market research, etc. For this tutorial, I will be using Mistral AI, but you could do the same with any LLM provider.

I have stored the `MISTRAL_AI_KEY` environment variable in my system. I will also be using *dotenv* module to store secrets.

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('MISTRAL_API_KEY')
if api_key is None:
    raise ValueError("MISTRAL_API_KEY not found in environment variables")
else:
    print("All Good to go!")

All Good to go!


In LangGraph, the interaction between user and AI agent is managed through a sequence of messages. The message types include following.
- AI Message: These are responses generated by the LLM
- System Message: Messages sent by the agent to provide context or additional instructions to the LLM
- Human Message: These are input from the user such as questions or commands.

## Q&A Agent

In this Q&A agent, you will use LLM to provide answers to user questions and also call external API using tool node to fetch real time information.
1. First you will accept user input
2. You will pass user question to the LLM. LLM can directly respond or trigger tool nodes if user is asking for current information.
3. Tool nodes are used to find the latest information.
4. Once the response has been generated, it is returned to the user.

In [2]:
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, MessagesState, START, END

model = ChatMistralAI(model='mistral-large-latest', api_key=api_key)

def call_llm(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages[-1].content)
    return {'messages': [response]}

workflow = StateGraph(MessagesState)
workflow.add_node('call_llm', call_llm)
workflow.add_edge(START, 'call_llm')
workflow.add_edge('call_llm', END)

app = workflow.compile()

input_message = {
    'messages': [('human', 'What is the capital of Kenya?')]
}
for chunk in app.stream(input_message, stream_mode='values'):
    chunk['messages'][-1].pretty_print()

================================ Human Message =================================

What is the capital of Kenya?
================================== Ai Message ==================================

The capital of Kenya is **Nairobi**. It is the largest city in Kenya and serves as the country's political, economic, and cultural hub.


The `call_llm` function sends the user input to the LLM model and returns the generated response.
Next, we want to be able to handle continuous user input with multiple question answers in a single session.

In [3]:
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, MessagesState, START, END

model = ChatMistralAI(model='mistral-large-latest', api_key=api_key)

def call_llm(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages[-1].content)
    return {'messages': [response]}

workflow = StateGraph(MessagesState)
workflow.add_node('call_llm', call_llm)
workflow.add_edge(START, 'call_llm')
workflow.add_edge('call_llm', END)

app = workflow.compile()

def interact_with_agent():
    while True: 
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Ending the conversation")
            break
        input_message = {
            'messages': [('human', user_input)]
        }
        
        for chunk in app.stream(input_message, stream_mode='values'):
            chunk['messages'][-1].pretty_print()
# interact_with_agent()

## Tools

LLM is great at generating responses based on language understanding, however, it has limitations like it lacks real-time information and cannot perform specific tasks like calling an API or running calculations. Tools allow an AI agent to fetch real-time data, perform specific tasks and retrieve information from databases or external APIs. `ToolNode` is used for calling external tools and integrate it into existing agent.

LangChain provides a convenient way to define tools using the `@tool` decorator which turns any Python function into callable tools. In below code, `@tool` decorator makes the function as a tool that can be used within LangGraph. The `get_weather` function takes location as input and returns the weather for that location.

In [4]:
from langchain_core.tools import tool

@tool
def get_weather(location: str):
    """
    Fetch the current weather for a specific location
    """
    weather_data = {
        "San Francisco": "It's 60 degrees and foggy.",
        "New York": "It's 90 degrees and sunny.",
        "London": "It's 70 degrees and cloudy."
    }
    return weather_data.get(location, "Weather information is unavailable for this location.")

Once you've tool for fetching the weather information, you need to integrate it into the agent. LangGraph provides a node type called `ToolNode` which is responsible for calling external tools. It takes a list of tools as input.

In [5]:
from langgraph.prebuilt import ToolNode
tool_node = ToolNode([get_weather])

In [6]:
model = ChatMistralAI(model='mistral-large-latest', api_key=api_key).bind_tools([get_weather])

def call_llm(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages[-1].content)
    if response.tool_calls:
        tool_result = tool_node.invoke({'messages': [response]})
        tool_message = tool_result['messages'][-1].content
        response.content += f"\nTool Result: {tool_message}"
        return {'messages': [response]}

workflow = StateGraph(MessagesState)
workflow.add_node('call_llm', call_llm)
workflow.add_edge(START, 'call_llm')
workflow.add_edge('call_llm', END)

app = workflow.compile()

In [7]:
def interact_with_agent():
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Ending the conversation")
            break
        input_message = {
            "messages": [("human", user_input)]
        }
        for chunk in app.stream(input_message, stream_mode='values'):
            chunk['messages'][-1].pretty_print()

In [8]:
# interact_with_agent()

When user asks a question, LLM node processes the input and it generates a response. If the response contains a *tool call*, the graph triggers the `ToolNode` and calls specified tool and returns the result.

## Error Handling in Tools

LangGraph provides built-in error handling for tool calls. If something goes wrong during tool execution, LangGraph will handle the error and return a meaningful message to the user. You can customize the error handling by configuring the `ToolNode` to handle or propagate errors.

```python
# ToolNode with error handling disabled (propagating errors to the user)
tool_node = ToolNode([get_weather], handle_tool_errors=False)
```

If the weather tool encounters an error, the agent will let the user konw that something went wrong rather than silently handling the error.

## Understanding Tool call

In this case, you will mimic tool calling for getting user profile. It includes following steps.
1. Define your tool using a Python function with `@tool` decorator. In this case, I have defined `get_user_profile` function which can retrieve user profile using `user_id`.
2. Next, set up `ToolNode` for calling the `get_user_profile` tool when AI agent asks for it.
3. In order to mimic tool calling, define `AIMessage` which tells the AI agent to call specific tool and provides required input.
4. Next, set up the `StateGraph`. In this case, the `StateGraph` must have `messages` key.
5. Invoke the tool using `ToolNode` to process the state. The `ToolNode` will look at the last message in the state, find the tool call and execute the tool.

In [9]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage

# 1. Define the tool
@tool
def get_user_profile(user_id: str):
    """
    Fetch the profile of a user by user ID
    """
    user_data = {
        "101": {"name": "Alice", "age": 30, "location": "New York"},
        "102": {"name": "Bob", "age": 25, "location": "San Francisco"}
    }
    return user_data.get(user_id, "User profile not found.")

# 2. Setup ToolNode
tools = [get_user_profile]
tool_node = ToolNode(tools)

# 3. Set up AIMessage for tool calling
message_with_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "get_user_profile",
            "args": {"user_id": "101"},
            "id": "tool_call_id",
            "type": "tool_call"
        }
    ]
)

# 4. Set up StateGraph
state = {
    "messages": [message_with_tool_call]
}

# 5. Invoke the ToolNode with state
result = tool_node.invoke(state)
print(result)

{'messages': [ToolMessage(content='{"name": "Alice", "age": 30, "location": "New York"}', name='get_user_profile', tool_call_id='tool_call_id')]}


## Adding Memory

Short-term memory helps an agent maintain context during a conversation within a session but not across multiple sessions, making it more coherent.

### Agent without Memory

In [10]:
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, MessagesState, START, END
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")

model = ChatMistralAI(model='mistral-large-latest', api_key=api_key)

def call_llm(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages[-1].content)
    return {'messages': [response]}

workflow = StateGraph(MessagesState)
workflow.add_node('call_llm', call_llm)
workflow.add_edge(START, 'call_llm')
workflow.add_edge('call_llm', END)

app = workflow.compile()

def interact_with_agent():
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Ending the conversation.")
            break
        input_message = {
            'messages': [('human', user_input)]
        }
        for chunk in app.stream(input_message, stream_mode='values'):
            chunk['messages'][-1].pretty_print()

In [11]:
# interact_with_agent()

You:  My name is Jenny Zim.


================================ Human Message =================================

My name is Jenny Zim.
================================== Ai Message ==================================

Nice to meet you, Jenny Zim! How can I assist you today? 😊


You:  What is my name?


================================ Human Message =================================

What is my name?
================================== Ai Message ==================================

I don’t have access to your name unless you’ve shared it with me in our conversation. If you’d like, you can tell me your name, and I’ll remember it for this chat! 😊

What should I call you?


You:  quit


Ending the conversation.


### Agent with Short-term Memory

With short-term meomry, the agent can remember the conversation during the session but will forget everything once the session ends. LangGraph offers `MemorySaver` for implementing short-term memory easily.

In [12]:
from langgraph.checkpoint.memory import MemorySaver

def call_llm(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    return {'messages': [response]}

memory = MemorySaver()

workflow = StateGraph(MessagesState)
workflow.add_node('call_llm', call_llm)
workflow.add_edge(START, 'call_llm')
workflow.add_edge('call_llm', END)
app_with_memory = workflow.compile(checkpointer=memory)

def interact_with_agent_with_memory():
    thread_id = 'session_1'
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Ending the conversation.")
            break
        input_message = {
            'messages': [('human', user_input)]
        }
        config = {'configurable': {'thread_id': thread_id}}
        for chunk in app_with_memory.stream(input_message, config=config, stream_mode='values'):
            chunk['messages'][-1].pretty_print()

In [13]:
# interact_with_agent_with_memory()

You:  My name is Jenny Zim.


================================ Human Message =================================

My name is Jenny Zim.
================================== Ai Message ==================================

Nice to meet you, Jenny Zim! How can I assist you today? 😊


You:  What is my name?


================================ Human Message =================================

What is my name?
================================== Ai Message ==================================

Your name is **Jenny Zim**—nice to "meet" you again! 😊 Let me know if there’s anything I can help with.


You:  quit


Ending the conversation.


In [14]:
from langchain_core.messages import HumanMessage
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
# Define a config with a thread_id
config = {"configurable": {"thread_id": "conversation_1"}}
model = ChatMistralAI(model='mistral-medium-latest')
memory = MemorySaver()

# Define the structure of the state
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Define the logic
def chatbot(state: State):
    return {"messages": [model.invoke(state["messages"])]}

# Build the Graph
workflow = StateGraph(State)
workflow.add_node("chatbot", chatbot)
workflow.add_edge(START, "chatbot")
workflow.add_edge("chatbot", END)

app_with_memory = workflow.compile(checkpointer=memory)
# First interaction
user_input = "Hi, my name is Alex."
result = app_with_memory.invoke({"messages": [HumanMessage(content=user_input)]}, config)
print(result["messages"][-1].content)

# Second interaction (memory recalls the name)
user_input = "What is my name?"
result = app_with_memory.invoke({"messages": [HumanMessage(content=user_input)]}, config)
print(result["messages"][-1].content)


Hi Alex! 😊 Nice to meet you! How’s your day going so far? Is there anything I can help you with—whether it’s answering questions, brainstorming ideas, or just chatting? Let me know! 🚀

(Also, fun fact: "Alex" is a great name—it’s got that classic yet versatile vibe! 👌)
Your name is **Alex**—just like you introduced yourself! 😄

(Though if you ever want to go by a different name or nickname, just let me know—I’m happy to adjust!) 👋✨


With `MemorySaver`, you can store the state of the conversation within the session. LangGraph saves checkpoints (snapshots of the conversation state) at every step and linked to a thread ID which simulates a session. Once the session ends, memory is discarded.

### Memory Across multiple sessions

LangGraph uses checkpointers and thread IDs to store the state of the conversation at every interaction (super-step). Thread IDs uniquely identify a session or conversation, allowing the agent to restore the conversation from a previous checkpoint when the same thread ID is provided. With this information, you can modify existing agent to persist memory between different sessions by using thread IDs to link conversations. When the program ends the data in RAM will be lost since session are in-memory at the moment.

In [15]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, MessagesState, START, END
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")

model = ChatMistralAI(model='mistral-medium-latest', api_key=api_key)

def call_llm(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    return {'messages': [response]}

memory = MemorySaver()

workflow = StateGraph(MessagesState)
workflow.add_node('call_llm', call_llm)
workflow.add_edge(START, 'call_llm')
workflow.add_edge('call_llm', END)
app_with_memory = workflow.compile(checkpointer=memory)

def interact_with_agent_across_sessions():
    while True:
        thread_id = input("Enter thread ID (or 'new' for a new session): ")
        if thread_id.lower() in ['quit', 'exit', 'end_session']:
            print("Ending the conversation.")
            break
        if thread_id.lower() == 'new':
            thread_id = f"session_{os.urandom(4).hex()}"
        while True:
            user_input = input("You: ")
            if user_input.lower() in ['exit', 'quit']:
                print(f"Ending the session {thread_id}.")
                break
            input_message = {
                'messages': [('human', user_input)]
            }
            config = {'configurable': {'thread_id': thread_id}}
            for chunk in app_with_memory.stream(input_message, config=config, stream_mode='values'):
                chunk['messages'][-1].pretty_print()

In [16]:
# interact_with_agent_across_sessions()

Enter thread ID (or 'new' for a new session):  new
You:  My name Jack Daniel.


================================ Human Message =================================

My name Jack Daniel.
================================== Ai Message ==================================

Nice to meet you, **Jack Daniel**! That’s a strong and classic name—like the famous whiskey (though I’m sure you’ve heard that before).

How can I help you today? Need advice, trivia, or just a fun chat? Let me know! 🥃 (Or not—your call.)


You:  What is my last name?


================================ Human Message =================================

What is my last name?
================================== Ai Message ==================================

Based on your introduction, your last name is **Daniel**—so your full name is **Jack Daniel**.

(Though if you're referring to the whiskey brand, it’s *Jack Daniel’s*, with an apostrophe, named after its founder, Jasper Newton "Jack" Daniel.)

Need help with something else? 😊


You:  quit


...

================================ Human Message =================================

What is my name?
================================== Ai Message ==================================

I don’t have access to personal information about you unless you share it with me! If you’d like, you can tell me your name, and I’ll be happy to use it in our conversation. 😊

(Or if this is a fun riddle—let me know, and I’ll play along!)


You:  quit


Ending the session session_b370185d.


Enter thread ID (or 'new' for a new session):  quit


Ending the conversation.


LangGraph saves the state of the conversation at every interaction as a checkpoint with each checkpoint containing the conversation context. Every session is associated with a thread ID. When the same thread ID is reused, LangGraph restores the conversation context from the last checkpoint associated with that thread.

Sometimes, it's necessary for the agent to remember certain user information like settings, personal data across all sessions even when a new thread is started. For this, LangGraph provides the `MemoryStore`. `MemoryStore` allows the agent to store information that can be shared across different sessions for the same user. For example, the agent could store user's preferred language or settings which would persist across all future conversations regardlesss of the session ID.

In [17]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, MessagesState, START, END
from dotenv import load_dotenv
import os

from langgraph.store.memory import InMemoryStore
import uuid

load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")

model = ChatMistralAI(model='mistral-medium-latest', api_key=api_key)
in_memory_store = InMemoryStore()

def store_user_info(state: MessagesState, config, *, store=in_memory_store):
    user_id = config['configurable']['user_id']
    namespace = (user_id, 'memories')
    # Create memory based on the conversation memory_id
    memory_id = str(uuid.uuid4())
    memory = {'user_name': state['user_name']}
    # Save the memory to the in-memory store
    store.put(namespace, memory_id, memory)
    return {'messages': ['User information saved.']}

def retrieve_user_info(state: MessagesState, config, *, store=in_memory_store):
    user_id = config['configurable']['user_id']
    namespace = (user_id, 'memories')
    memories = store.search(namespace)
    if memories:
        info = f"Hello {memories[-1].value['user_name']}, welcome back!"
    else:
        info = "I don't have any information about you yet."
    return {'messages': [info]}

def call_model(state: MessagesState, config):
    last_message = state['messages'][-1].content.lower()
    if 'remember my name' in last_message:
        user_name = last_message.split('remember my name is')[-1].strip()
        state['user_name'] = user_name
        return store_user_info(state, config)
    if "what's my name" in last_message or "what is my name" in last_message:
        # Retrieve user's name from memory
        return retrieve_user_info(state, config)
    # Default LLM response for other inputs
    return {"messages": ["I didn't understand your request."]}

workflow = StateGraph(MessagesState)
workflow.add_node("call_model", call_model)
workflow.add_edge(START, "call_model")
workflow.add_edge("call_model", END)

app_with_memory = workflow.compile(checkpointer=MemorySaver(), store=in_memory_store)

# Simulate sessions
def simulate_sessions():
    config = {"configurable": {
        "thread_id": "session_1",
        "user_id": "user_123"
    }}
    input_message = {
        "messages": [
            {"type": "user", "content": "Remember my name is Alice"}
        ]
    }

    for chunk in app_with_memory.stream(input_message, config=config, stream_mode="values"):
        chunk["messages"][-1].pretty_print()

    config = {"configurable": {
        "thread_id": "session_2",
        "user_id": "user_123"
    }}
    input_message = {
        "messages": [
            {"type": "user", "content": "What is my name?"}
        ]
    }

    for chunk in app_with_memory.stream(input_message, config=config, stream_mode="values"):
        chunk["messages"][-1].pretty_print()
    
simulate_sessions()

================================ Human Message =================================

Remember my name is Alice
================================ Human Message =================================

User information saved.
================================ Human Message =================================

What is my name?
================================ Human Message =================================

Hello alice, welcome back!


### Checkpointers

This is responsible for saving the state of a graph at each super-step in the workflow. Each checkpoint is a snapshot of the current graph state and contains crucial information like configuration, metadata and state values. A checkpoint is represented by a `StateSnapshot` object and contains following properties.
- `Config` contains configuration associated with the checkpoint including `thread_id` and optional `checkpoint_id`.
- `Metadata` provides details about the source of the checkpoint and the graph's progress at this point.
- `Values` represent the current state of the channels in the graph at the time the checkpoint was taken.
- `Next` is a tuple of the node names to execute next in the graph.
- `Tasks` is a tuple of `PregelTask` objects that contain information about the next tasks to execute in the graph. It also holds error data if an execution failed or was interrupted.

Each checkpoint represents the state of the graph at a specific super-step and can be replayed or updated.


In [18]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict

class State(TypedDict):
    foo: str
    bar: list[str]

def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State): 
    return {"foo": "b", "bar": ["a", "b"]}

workflow = StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge('node_b', END)

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"foo": "", "bar":[]}, config)

{'foo': 'b', 'bar': ['a', 'b']}

You can retrieve the latest state of the graph by calling `graph.get_state()`.

In [19]:
config = {"configurable": {"thread_id": "1"}}
latest_state = graph.get_state(config)
print(latest_state.values)

{'foo': 'b', 'bar': ['a', 'b']}


In [20]:
# Get state history
config = {"configurable": {"thread_id": "1"}}
state_history = graph.get_state_history(config)
for snapshot in state_history:
    print(snapshot.values)

{'foo': 'b', 'bar': ['a', 'b']}
{'foo': 'a', 'bar': ['a']}
{'foo': '', 'bar': []}
{}


### InMemoryStore
This allows you to persist information across different threads and sessions. While checkpointers are tied to a specific session, the memory store can retain user  informaiton, preferences and history between sessions.
- In `InMemoryStore`, memories are saved using a namespace which typically includes a `user_id` to uniquely identify the memory.
- It uses `put()` to store a memory and `search()` function to retrieve from it.

In [23]:
from langgraph.store.memory import InMemoryStore
import uuid

in_memory_store = InMemoryStore()
# Define user namespace using (user_id + 'memories')
user_id = "1"
namespace_for_memory = (user_id, "memories")

# store food preferences
memory_id = str(uuid.uuid4())
memory = {"food_preference": "I like pizza"}
in_memory_store.put(namespace_for_memory, memory_id, memory)

# Retrieve from memories
memories = in_memory_store.search(namespace_for_memory)
print(memories[-1].dict())

{'namespace': ['1', 'memories'], 'key': '6cb5b0fa-85cc-4bca-bb8a-b238f7549b5a', 'value': {'food_preference': 'I like pizza'}, 'created_at': '2026-01-28T02:03:11.691429+00:00', 'updated_at': '2026-01-28T02:03:11.691438+00:00', 'score': None}


In practical applications, you will often use `checkpointers` for session-based memory and `InMemoryStore` for persistent memory across sessions.

In [24]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
## Initialize checkpointer and memory store
checkpointer = MemorySaver()
in_memory_store = InMemoryStore()

## Compile the graph with memory and checkpointers
graph = workflow.compile(checkpointer=checkpointer, store=in_memory_store)

## invoke the graph with thread_id and user_id config
config = {
            "configurable": {
                "thread_id": "session_1",
                "user_id": "1"
            }
         }
graph.invoke({"foo": ""}, config)

def update_memory(state: MemoryState, config: RunnableConfig, *, store: BaseStore):
    user_id = config['configurable']['user_id']
    namespace = (user_id, "memories")
    # Store a memory
    memory_id = str(uuid.uuid4())
    store.put(namespace, memory_id, {"favorite_food": "pizza"})
    # Retrieve stored memories 
    memories = store.search(namespace)
    return {"message": [f"I remember you like {memories[-1].value['favorite_food']}"]}
    

NameError: name 'MemoryState' is not defined